### Convert coefficients of parameterization from F77 to F90

Coefficients are originally declared in the offline code, which is in F77.  In order to use these in the global model, they need to be re-expressed in F90, before being inserted into the global model source code.

In [21]:
import os
import io
import collections
import re
import itertools

import numpy as np
import pandas as pd
import xarray as xr

In [192]:
dir_f77 = '/nuwa_cluster/home/jackyu/radiation/clirad/LW/src'
files_trans = {'h2o': 'h2o_tran3.F',
               'co2': 'co2_tran4.F',
               'o3': 'o3_tran3.F'}

band1_trans = dict(h2o=[1, 2, 8], co2=[1], o3=[1])
band2_trans = dict(h2o=[1, 2, 3], co2=[1, 2, 3], o3=[1, 2, 3])
num_pressures_trans = dict(h2o=26, co2=26, o3=26)
num_amounts_trans = dict(h2o=31, co2=30, o3=21)

In [221]:
gas = 'o3'

In [233]:
def load_gas_trans(gas='h2o', dir_f77='./'):
    '''
    Get gas transmission values from CLIRAD-LW offline model and
    store them in xarray.DataArray
    
    Parameters
    ----------
    gas: name of the gas: h2o, co2, or o3
    dir_f77: directory containing F77 files containing transmissions
    '''
    with open(os.path.join(dir_f77, files_trans[gas]), 
              encoding='utf-8', mode='r') as file:
        content = file.read()
        
    lines_data = (l for l in content.split('\n') if '&' in l)
    str_data = ''.join(lines_data)
    str_data = str_data.replace('&', '').replace('/', '')\
               .replace(',', '')
    
    df = pd.read_csv(io.StringIO(str_data), sep=r'\s+', header=None)
    data = np.squeeze(df).values
    
    num_band1s = len(band1_trans[gas])
    num_band2s = len(band2_trans[gas])
    num_pressures = num_pressures_trans[gas]
    num_amounts = num_amounts_trans[gas]
    
    data_ibs = data.reshape(num_band1s, int(data.shape[0] / num_band1s))
    
    pressure = range(1, num_pressures + 1)
    amount = range(1, num_amounts + 1)

    list_da_band1s = [xr.DataArray(data_ib.reshape(num_pressures, 
                                                   num_band2s, 
                                                   num_amounts),
                                   dims=['pressure', 'band2', 'amount'],
                                   coords=[pressure, 
                                           band2_trans[gas], 
                                           amount]) 
                      for data_ib in data_ibs]

    da = xr.concat(list_da_band1s, dim='band1')
    da.coords['band1'] = ('band1', band1_trans[gas])
    return da

In [247]:
da_h2o = load_gas_trans(gas='h2o', dir_f77=dir_f77)

In [236]:
da_co2 = load_gas_trans(gas='co2', dir_f77=dir_f77)

In [240]:
da_o3 = load_gas_trans(gas='o3', dir_f77=dir_f77)

In [255]:
writedata = da_o3.sel(pressure=3, band2=3, band1=1).values.copy()

In [256]:
writedata.resize((5, 5))

In [257]:
writedata

array([[ -2.43300000e-11,  -4.81500000e-11,  -9.45300000e-11,
         -1.82500000e-10,  -3.40000000e-10],
       [ -6.21500000e-10,  -1.00700000e-09,  -1.41500000e-09,
         -1.25300000e-09,   8.14300000e-10],
       [  6.26900000e-09,   1.18600000e-08,   2.60400000e-09,
         -4.06700000e-08,  -1.26400000e-07],
       [ -2.57900000e-07,  -4.32100000e-07,  -6.22900000e-07,
         -7.73200000e-07,  -8.27700000e-07],
       [ -7.75200000e-07,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00]])